The project is about merging 3 dataset Energy, Productions, and Fermate

In [ ]:
from scripts.getDataset import getEntireDataset
from scripts.plots import plot
import pandas as pd

pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False

dataset = getEntireDataset(306, 2023, 1)

plot(dataset)

Now I iterate for every machine to get the data from the 3 datasets and merge them into one dataset, removing the ones that are not useful for the analysis.

Now we can check for all the dataset and see what we can do with them

In [3]:
from scripts.plots import plot
from scripts.getDataset import getEntireDataset, forEveryMachine, cleanDataset
import pandas as pd

DEBUG = False
pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False

most_interesting = []

def fn(machineId, year, month):
    dataset = getEntireDataset(machineId, year, month, False)

    if dataset.empty:
        return

    if dataset["EnergyConsumption"].dropna().eq(0).all():
        if DEBUG:
            print("Skipping as EnergyConsumption is all 0")
        return
    if dataset["Fermate"].dropna().eq(0).all():
        if DEBUG:
            print("Skipping as Fermate is all 0")
        return

    EC_na = dataset["EnergyConsumption"].isna().sum()
    EC_len = dataset["EnergyConsumption"].shape[0]

    ST_na = dataset["Fermate"].isna().sum()
    ST_len = dataset["Fermate"].shape[0]

    energyRate = EC_na/ EC_len
    stopsRate = ST_na/ ST_len

    if energyRate > 0.5 or stopsRate > 0.5:
        if DEBUG:
            print("Skipping as too many NaN values")
            print("\tEnergy Rate:", energyRate)
            print("\tStops Rate:", stopsRate)
        return

    most_interesting.append((cleanDataset(dataset), machineId, year, month))

forEveryMachine(fn)

print("\n\n\nThe most interesting are the followings:")
for dataset, machineId, year, month in most_interesting:
    dataset.to_csv(f"dataset/results/it1/id-{machineId}_{year}-{month}.csv")
    print(f"- Machine {machineId} - {year}/{month}")

for dataset, machineId, year, month in most_interesting:
    plot(dataset, machineId, year, month)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107e7c5d0>>
Traceback (most recent call last):
  File "/Users/leonardomigliorelli/.pyenv/versions/3.11.6/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

It took 20 minutes to compute everything but now I want to look at the lifetime of the machines and see if I can find something interesting, merging the data into one file for machine

In [4]:
from scripts.getSingleDataset.utils import getCleanDataset
import pandas as pd
import os

files = [p for p in os.listdir("dataset/results/it1") if p.endswith(".csv")]

compete_dataset = {}

for path in files:
    # dataset.to_csv(f"dataset/results/it1/id-{machineId}_{year}-{month}.csv")

    splitted_filename = [p.split("-") for p in path.replace(".csv", "").split("_")]

    machineId = int(splitted_filename[0][1])
    year = int(splitted_filename[1][0])
    month = int(splitted_filename[1][1])

    d = getCleanDataset(f"dataset/results/it1/id-{machineId}_{year}-{month}.csv")

    # plot(d)
    value = [d]

    if machineId in compete_dataset:
        old_data = compete_dataset.get(machineId)
        
        for old in old_data:
            value.append(old)
        

    compete_dataset.update({machineId: value})

for machineId in compete_dataset.keys():
    d = pd.concat(compete_dataset.get(machineId))

    d.to_csv(f"dataset/results/it2/id-{machineId}.csv", mode="w", index=False)

Now I use the pearson correlation to see the correlation between datsets

In [5]:
import os
from scripts.getSingleDataset.utils import getCleanDataset
from scipy import stats
from scripts.plots import plot

files = [p for p in os.listdir("dataset/results/it2") if p.endswith(".csv")]

STR = "\t\t{:0.2f}\n\t\tP-Value:{:0.2f}"

def is_correlation_usable(data):
    computing_data = data.dropna()

    print(f"Machine {machineId}")
    # Correlazione tra Energia e Produzioni
    energy_productions_corr, energy_productions_pvalue = stats.pearsonr(
        computing_data["EnergyConsumption"], computing_data["Productions"]
    )
    print("\tCorrelation between Energy Consumption and Productions")
    print(STR.format(energy_productions_corr, energy_productions_pvalue))
    if energy_productions_pvalue <= 0.05:
        print("\t\tSignificant")

    # Correlazione tra Energia e Fermate
    energy_stops_corr, energy_stops_pvalue = stats.pearsonr(
        computing_data["EnergyConsumption"], computing_data["Fermate"]
    )
    print("\tCorrelation between Energy Consumption and Stops")
    print(STR.format(energy_stops_corr, energy_stops_pvalue))
    if energy_stops_pvalue <= 0.05:
        print("\t\tSignificant")

    # Correlazione tra Produzioni e Fermate
    productions_stops_corr, productions_stops_pvalue = stats.pearsonr(
        computing_data["Productions"], computing_data["Fermate"]
    )
    print("\tCorrelation between Productions and Stops")
    print(STR.format(productions_stops_corr, productions_stops_pvalue))
    if productions_stops_pvalue <= 0.05:
        print("\t\tSignificant")

    return (
        energy_productions_pvalue <= 0.05
        or energy_stops_pvalue <= 0.05
        or productions_stops_pvalue <= 0.05
    )

usable_datasets = []
for file in files:
    machineId = int(file.replace(".csv", "").split("-")[1])

    data = getCleanDataset(f"dataset/results/it2/{file}")

    if is_correlation_usable(data):
        usable_datasets.append((machineId, data))

print("Usable datasets: ", len(usable_datasets))

for machineId, d in usable_datasets:
    d.to_csv(f"dataset/results/it3/id-{machineId}.csv", mode="w", index=False)
    print("- Macine", machineId)

for machineId, d in usable_datasets:
    plot(d, machineId)

Machine 304
	Correlation between Energy Consumption and Productions
		0.59
		P-Value:0.00
		Significant
	Correlation between Energy Consumption and Stops
		0.14
		P-Value:0.50
	Correlation between Productions and Stops
		0.36
		P-Value:0.08
Machine 310
	Correlation between Energy Consumption and Productions
		0.27
		P-Value:0.40
	Correlation between Energy Consumption and Stops
		0.43
		P-Value:0.16
	Correlation between Productions and Stops
		0.29
		P-Value:0.37
Machine 515
	Correlation between Energy Consumption and Productions
		0.13
		P-Value:0.26
	Correlation between Energy Consumption and Stops
		0.08
		P-Value:0.52
	Correlation between Productions and Stops
		0.38
		P-Value:0.00
		Significant
Machine 313
	Correlation between Energy Consumption and Productions
		0.19
		P-Value:0.44
	Correlation between Energy Consumption and Stops
		0.24
		P-Value:0.32
	Correlation between Productions and Stops
		0.24
		P-Value:0.32
Machine 307
	Correlation between Energy Consumption and Producti

/var/folders/8b/pr_5nfyx79x9nz4pw899j4wm0000gn/T/ipykernel_19808/3917665905.py:24: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  energy_stops_corr, energy_stops_pvalue = stats.pearsonr(
/var/folders/8b/pr_5nfyx79x9nz4pw899j4wm0000gn/T/ipykernel_19808/3917665905.py:33: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  productions_stops_corr, productions_stops_pvalue = stats.pearsonr(


In [7]:
import os
from scripts.getSingleDataset.utils import getCleanDataset

files = [p for p in os.listdir("dataset/results/it2") if p.endswith(".csv")]

tot = 0
for file in files:
    d = getCleanDataset(f"dataset/results/it2/{file}")
    tot += d.shape[0]

print(tot)

2389
